- Extraction of sentence

In [1]:
import pandas as pd
import Extractor
import sqlite3
df = pd.read_excel('들,2,3_0.xlsx')
new_words = df[df.y == 1].iloc[:,1].drop_duplicates(keep='first')

In [2]:
conn = sqlite3.connect('Total_Ent.db')
df1 = pd.read_sql('SELECT head FROM head', conn)
conn.close()
conn = sqlite3.connect('HP.db')
df2 = pd.read_sql('SELECT head FROM head', conn)
conn.close()

In [3]:
head = df1.append(df2)

In [4]:
ext = Extractor.Ext(head)
head = ext.cleaning()

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107


In [5]:
sent = ext.extract_sent(new_words)

In [25]:
sent = pd.DataFrame.from_dict(sent, orient='index')

In [33]:
conn = sqlite3.connect('sent.db')
sent.to_sql('sent', conn)

- Extract PMI Dictionary

In [159]:
conn = sqlite3.connect('sent.db')
temp = pd.read_sql('SELECT * FROM sent', conn)

In [160]:
from collections import defaultdict
sent = defaultdict(lambda:0)
for _ in range(len(temp)):
    sent[temp['index'][_]] = temp['0'][_]

In [172]:
from soynlp.word import WordExtractor
from soynlp.tokenizer import MaxScoreTokenizer
from soynlp.vectorizer import sent_to_word_contexts_matrix
from soynlp.word import pmi as pmi_func
from soynlp.utils import most_similar
from soynlp.tokenizer import LTokenizer

pmi_dict = defaultdict(lambda:0)
word_extractor = WordExtractor()
words = {_:1.0 for _ in sent.keys()}
sentiment = pd.read_excel('sentiment.xlsx')
for k,v in sent.items():
    word_extractor.train([v])
    cohesions = word_extractor.all_cohesion_scores()
    l_cohesions = {word:score[0] for word, score in cohesions.items()}
    l_cohesions.update(words)
    tokenizer = LTokenizer(l_cohesions)
    x, idx2vocab = sent_to_word_contexts_matrix([v], windows = 3, min_tf = 10, tokenizer=tokenizer, dynamic_weight = False, verbose = True)
    pmi, px, py = pmi_func(x, min_pmi = 0, alpha = 0.0, beta = 0.75)
    vocab2idx = {vocab:idx for idx, vocab in enumerate(idx2vocab)}
    query = vocab2idx[k]
    submatrix = pmi[query,:].tocsr() # get the row of query
    contexts = submatrix.nonzero()[1] # nonzero() return (rows, columns)
    pmi_i = submatrix.data
    most_relateds = [(idx, pmi_ij) for idx, pmi_ij in zip(contexts, pmi_i)]
    most_relateds = sorted(most_relateds, key=lambda x:-x[1])[:10]
    most_relateds = [(idx2vocab[idx], pmi_ij) for idx, pmi_ij in most_relateds if len(idx2vocab[idx]) > 1]
    for _ in most_relateds:
        pmi_dict[_[0]] += 1
#     similar = [_[0] for _ in most_similar(k, pmi, vocab2idx, idx2vocab,topk=10) if len(_[0]) > 1]
#     P_N = {'p':0, 'n':0}
#     for _ in similar:
#         if _ in ' '.join(sentiment.iloc[:,1]) :
#             if [sentiment[sentiment[0] == _]['P/N'] == 1][0].bool():
#                 P_N['p'] += 4
#             elif [sentiment[sentiment[0] == _]['P/N'] == 2][0].bool():
#                 P_N['p'] += 2
#             elif [sentiment[sentiment[0] == _]['P/N'] == -1][0].bool():
#                 P_N['n'] += 3
#             elif [sentiment[sentiment[0] == _]['P/N'] == -2][0].bool():
#                 P_N['n'] += 1
#             else:
#                 pass
#     if P_N['p'] < P_N['n']:
#         pmi_dict[k] = 'negative'
#     else:
#         pmi_dict[k] = 'positive'
            
            
#         pmi_dict[_] += 1

training was done. used memory 0.572 Gb0.572 Gb
all cohesion probabilities was computed. # words = 627
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.572 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.572 Gb
  - (word, context) matrix was constructed. shape = (112, 112)                    
  - done
training was done. used memory 0.572 Gb0.572 Gb
all cohesion probabilities was computed. # words = 864
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.572 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.572 Gb
  - (word, context) matrix was constructed. shape = (52, 52)                    
  - done
training was done. used memory 0.572 Gb0.572 Gb
all cohesion probabilities was computed. # words = 1001
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.572 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.572 Gb
  - (word, context) matrix was constructed. shape = (41, 41)    

training was done. used memory 0.577 Gb0.577 Gb
all cohesion probabilities was computed. # words = 8300
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.577 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.577 Gb
  - (word, context) matrix was constructed. shape = (21, 21)                    
  - done
training was done. used memory 0.577 Gb0.577 Gb
all cohesion probabilities was computed. # words = 8438
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.577 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.577 Gb
  - (word, context) matrix was constructed. shape = (69, 69)                    
  - done
training was done. used memory 0.577 Gb0.578 Gb
all cohesion probabilities was computed. # words = 9020
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.577 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.577 Gb
  - (word, context) matrix was constructed. shape = (301, 301)  

training was done. used memory 0.585 Gb0.585 Gb
all cohesion probabilities was computed. # words = 18652
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.585 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.585 Gb
  - (word, context) matrix was constructed. shape = (27, 27)                    
  - done
training was done. used memory 0.585 Gb0.585 Gb
all cohesion probabilities was computed. # words = 18673
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.585 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.585 Gb
  - (word, context) matrix was constructed. shape = (16, 16)                    
  - done
training was done. used memory 0.585 Gb0.585 Gb
all cohesion probabilities was computed. # words = 18731
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.585 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.585 Gb
  - (word, context) matrix was constructed. shape = (51, 51) 

training was done. used memory 0.586 Gb0.586 Gb
all cohesion probabilities was computed. # words = 22296
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.586 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.586 Gb
  - (word, context) matrix was constructed. shape = (67, 67)                    
  - done
training was done. used memory 0.586 Gb0.586 Gb
all cohesion probabilities was computed. # words = 22352
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.586 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.586 Gb
  - (word, context) matrix was constructed. shape = (62, 62)                    
  - done
training was done. used memory 0.586 Gb0.586 Gb
all cohesion probabilities was computed. # words = 22472
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.586 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.586 Gb
  - (word, context) matrix was constructed. shape = (82, 82) 

training was done. used memory 0.586 Gb0.593 Gb
all cohesion probabilities was computed. # words = 26452
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.586 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.586 Gb
  - (word, context) matrix was constructed. shape = (60, 60)                    
  - done
training was done. used memory 0.597 Gb0.601 Gb
all cohesion probabilities was computed. # words = 30852
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.599 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.599 Gb
  - (word, context) matrix was constructed. shape = (1856, 1856)                    
  - done
training was done. used memory 0.602 Gb0.602 Gb
all cohesion probabilities was computed. # words = 30876
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.602 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.602 Gb
  - (word, context) matrix was constructed. shape = (69, 

...

all cohesion probabilities was computed. # words = 158997
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.576 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.576 Gb
  - (word, context) matrix was constructed. shape = (59, 59)                    
  - done
training was done. used memory 0.576 Gb0.576 Gb
all cohesion probabilities was computed. # words = 159137
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.576 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.576 Gb
  - (word, context) matrix was constructed. shape = (160, 160)                    
  - done
training was done. used memory 0.576 Gb0.576 Gb
all cohesion probabilities was computed. # words = 159142
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.576 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.576 Gb
  - (word, context) matrix was constructed. shape = (21, 21)                    
  - done
training was d

training was done. used memory 0.577 Gb0.577 Gb
all cohesion probabilities was computed. # words = 160791
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.577 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.577 Gb
  - (word, context) matrix was constructed. shape = (60, 60)                    
  - done
training was done. used memory 0.577 Gb0.577 Gb
all cohesion probabilities was computed. # words = 160836
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.577 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.577 Gb
  - (word, context) matrix was constructed. shape = (42, 42)                    
  - done
training was done. used memory 0.577 Gb0.577 Gb
all cohesion probabilities was computed. # words = 160878
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.577 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.577 Gb
  - (word, context) matrix was constructed. shape = (54, 5

training was done. used memory 0.562 Gb0.562 Gb
all cohesion probabilities was computed. # words = 164308
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.562 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.562 Gb
  - (word, context) matrix was constructed. shape = (242, 242)                    
  - done
training was done. used memory 0.562 Gb0.562 Gb
all cohesion probabilities was computed. # words = 164375
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.562 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.562 Gb
  - (word, context) matrix was constructed. shape = (65, 65)                    
  - done
training was done. used memory 0.562 Gb0.562 Gb
all cohesion probabilities was computed. # words = 164441
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.562 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.562 Gb
  - (word, context) matrix was constructed. shape = (100

training was done. used memory 0.564 Gb0.564 Gb
all cohesion probabilities was computed. # words = 166921
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.564 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.564 Gb
  - (word, context) matrix was constructed. shape = (928, 928)                    
  - done
training was done. used memory 0.564 Gb0.564 Gb
all cohesion probabilities was computed. # words = 166923
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.564 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.564 Gb
  - (word, context) matrix was constructed. shape = (62, 62)                    
  - done
training was done. used memory 0.564 Gb0.564 Gb
all cohesion probabilities was computed. # words = 167027
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.564 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.564 Gb
  - (word, context) matrix was constructed. shape = (86,

training was done. used memory 0.565 Gb0.565 Gb
all cohesion probabilities was computed. # words = 169395
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.565 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.565 Gb
  - (word, context) matrix was constructed. shape = (74, 74)                    
  - done
training was done. used memory 0.565 Gb0.565 Gb
all cohesion probabilities was computed. # words = 169655
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.565 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.565 Gb
  - (word, context) matrix was constructed. shape = (374, 374)                    
  - done
training was done. used memory 0.565 Gb0.565 Gb
all cohesion probabilities was computed. # words = 169744
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.557 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.557 Gb
  - (word, context) matrix was constructed. shape = (124

In [175]:
temp = sorted(pmi_dict.items(), key=lambda _:_[1], reverse=True)

In [180]:
sentiment = pd.DataFrame(temp)

In [181]:
sentiment.to_excel('sentiment.xlsx')

- Sentiment Analysis